### Installs,Imports & Corpus

In [ ]:
#Necessary installs
!pip install -q -U nltk
!pip install -q -U LM
!pip install -q -U Levenshtein 
 # Necessary imports
import nltk
import math
 #Tokenizer that divides a text into a list of sentences, by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences
nltk.download('punkt',quiet=True)
from nltk import sent_tokenize
from nltk import word_tokenize
from pprint import pprint
from collections import Counter
from nltk.util import ngrams
from nltk.corpus import brown
from sklearn.model_selection import train_test_split
from Levenshtein import distance as ld

     |████████████████████████████████| 1.5 MB 12.1 MB/s 
     |████████████████████████████████| 748 kB 46.9 MB/s 
     |████████████████████████████████| 110 kB 9.5 MB/s 
     |████████████████████████████████| 893 kB 17.9 MB/s 


In [ ]:
nltk.download('brown',quiet=True) # downloading the specific corpus
sents = brown.sents(categories = ['adventure','belles_lettres','editorial','fiction','news'])  # categories of brown corpus we selected(We didn't select the whole brown corpus because it needed over 1 hour to compile)
print('The chosen corpus is',len(sents),'sentences long') 

The chosen corpus is 23715 sentences long


### Question 1

In [ ]:
#Lowercasing our data
sents = list(map(lambda x: list(map(lambda y: y.lower(),x)),sents))
#Splitting our data to train,test
x_train , x_test = train_test_split(sents,train_size=0.8,test_size=0.2,random_state=1)
#Extra test set for a very specific purpose
x_test2=x_test[0:(len(x_test)//100)]

In [ ]:
#Flattening our x_train list so we can count tokens that appear < 10
def flatten(t):
    return [item for sublist in t for item in sublist]

flat_list = flatten(x_train)
count_flat_list = nltk.FreqDist(flat_list) 

In [ ]:
#Creating a list of words that appear < 10
tokens_to_remove=[]
for i in flat_list:
  if count_flat_list[i]<10:
    tokens_to_remove.append(i)

In [ ]:
#Replacing words that appear < 10 with special token "unk" / Takes approximately 10 minutes using Runtime>Change Runtime Type>GPU from colab's options !!
#For every sentence
for i in range(len(x_train)):
  #For every word in each sentence
  for j in range(len(x_train[i])):
    #Replace with unk if TRUE
    if x_train[i][j] in tokens_to_remove:
      x_train[i][j]='unk'

In [ ]:
# Initializing the counters using code provided to us from the slides
unigram_counter = Counter()
bigram_counter = Counter()
trigram_counter = Counter()

for sent in x_train:

    # Update the unigram counter
    unigram_counter.update([(gram,) for gram in ["<start>"] + sent+ ["<end>"]]) # <end> special token is saved in unigrams for our knseser ney implemetation
    
    # Update the bigram counter
    bigram_pad_sent = ["<start>"] + sent +  ['<end>']    
    bigram_counter.update([(gram1, gram2) for gram1, gram2 in zip(bigram_pad_sent, bigram_pad_sent[1:])])

    # Update the trigram counter
    trigram_pad_sent = ["<start>"]*2 + sent +  ['<end>']*2
    trigram_counter.update([(gram1, gram2, gram3) for gram1, gram2, gram3 in zip(trigram_pad_sent, trigram_pad_sent[1:], trigram_pad_sent[2:])]) 


# pprint(unigram_counter.most_common(5))   
# pprint(bigram_counter.most_common(5))
# pprint(trigram_counter.most_common(5))

In [ ]:
# bigram probability function (Calculating the log probability to better showcase the results instead of showing the probability values which are pretty small)
def biprob(w1,w2,a = 1): 
  alpha = a
  if type(w1) == str:
    w1 = w1.lower()
  if type(w2) == str:
    w2 = w2.lower()
  # Bigram prob + laplace smoothing
  bigram_prob = (bigram_counter[(w1, w2)] + alpha) / (unigram_counter[(w1,)] + alpha*vocabsize)
  #print("bigram_prob: {0:.3f} ".format(bigram_prob))

  bigram_log_prob = math.log2(bigram_prob)
  #print("bigram_log_prob: {0:.3f}".format(bigram_log_prob) )  
  return bigram_log_prob

In [ ]:
#Function for trigram probability (Calculating the log probability/Same as above )
def triprob(w1,w2,w3,a=1):
  alpha=a
  w1 = w1.lower()
  w2 = w2.lower()
  w3 = w3.lower()
  trigram_prob = (trigram_counter[(w1, w2,w3)] + alpha) / (bigram_counter[(w1,w2)] + alpha*vocabsize)
  trigram_log_prob = math.log2(trigram_prob)
  return trigram_log_prob

In [ ]:
# Flattening the list to get all the tokens
flat_list2 = flatten(x_train) 

In [ ]:
# Storing the unique tokens to form our vocabulary (from the train set)
vocabsize= len(set(flat_list2)) 
vocabtrain = set(flat_list2)

In [ ]:
#We replace the words that are not in the vocabulary with the special token unk
#For every sentence in the test dataset
for sent in range (len(x_test)) :
  #For every word in each sentence in the test dataset
  for w in range ( len(x_test[sent])) :
    #Replace with unk if TRUE
    if x_test[sent][w] not in vocabtrain:
      x_test[sent][w]='unk'

### Question 2



In [ ]:
# Bigram 1st method using Laplace smoohing(Alpha=1)
sum_prob = 0
bigram_cnt = 0
alpha = 1.0
#Adding start and end tokens to each of our sentences
for sent in x_test:
    sent = ['<start>'] + sent + ['<end>']

    # Iterate over the bigrams of the sentence
    for idx in range(1, len(sent)):
      #For every bi-gram combination in our sentence
        bigram_prob = (bigram_counter[(sent[idx-1], sent[idx])] +alpha) / (unigram_counter[(sent[idx-1],)] + alpha*vocabsize) # not including the probabilities of the special start token with word but including the special end token
        sum_prob += math.log2(bigram_prob)
        bigram_cnt += 1

HC = -sum_prob / bigram_cnt
perpl = math.pow(2,HC)
print("Cross Entropy: {0:.3f}".format(HC)) # calculation of cross entropy and perplexity of our bigram model with Laplace smoothing 
print("perplexity: {0:.3f}".format(perpl))

Cross Entropy: 7.789
perplexity: 221.243


In [ ]:
def prev(w):
  count = 0 
  for v in vocabtrain:
    if bigram_counter[(v,w)]>0:
      count += 1
  return count

In [ ]:
def next(w):
  count = 0 
  for v in vocabtrain:
    if bigram_counter[(w,v)]>0:
      count += 1
  return count

In [ ]:
# Bigram probability calculation using Kneser Ney smoothing (our implementation)
def biprob_w_kneser(w1,w2,tokens=flat_list2,d= 0.5):
  S = 0
  w1=w1.lower()
  w2=w2.lower()
  for voc in vocabtrain:
    if bigram_counter[(w1,voc)]==0:
      S += prev(voc)
  if bigram_counter[(w1,w2)]>0:
    a= (bigram_counter[(w1,w2)]-d)/(unigram_counter[(w1,)])                             
  else:
    a= (d/(unigram_counter[(w1,)]))*next(w1)*(prev(w2)/S) 
  return a

Bigram-Kneser Ney smoothing written to guarantee a smoother running time.
(Can be enabled manually)

In [ ]:
biprob_w_kneser('I', 'am')

0.0468586387434555

In [ ]:
biprob_w_kneser('The', 'Government')

0.0012841161363339572

In [ ]:
biprob_w_kneser('Soviet', 'Union')

0.2230769230769231

In [ ]:
# #Bigram 2nd method using using Kneser Ney smoothing 
# sum_prob = 0
# bigram_cnt = 0
# alpha = 1.0
# for sent in x_test:
#     sent = ['<start>'] + sent + ['<end>']

#     # Iterate over the bigrmas of the sentence
#     for idx in range(1, len(sent)):
#         bigram_prob = biprob_w_kneser(sent[idx-1], sent[idx])
#         sum_prob += bigram_prob
#         bigram_cnt += 1
#         #print(idx)

# HC = -sum_prob / bigram_cnt
# perpl = math.pow(2,HC)
# print("Cross Entropy: {0:.3f}".format(HC)) # calculating cross entropy and perplexity of our model using the second model to compare the results 
# print("perplexity: {0:.3f}".format(perpl)) # with the two different smoothing techniques

In [ ]:
#Trigram 1st implementation using Laplace smoothing (therefore a=1.0) 
sum_prob = 0
trigram_cnt = 0

for sent in x_test:
    sent = ['<start>'] + ['<start>'] + sent + ['<end>'] + ['<end>']

    for idx in range(2,len(sent)-1):
        trigram_prob = (trigram_counter[(sent[idx-2],sent[idx-1], sent[idx])] +alpha) / (bigram_counter[(sent[idx-2],sent[idx-1])] + alpha*vocabsize)  # not including the probabilities of the special start token with word but including the special end token
        sum_prob += math.log2(trigram_prob)
        trigram_cnt+=1

HC = -sum_prob / trigram_cnt
perpl = math.pow(2,HC)
print("Cross Entropy: {0:.3f}".format(HC)) # calculation cross entropy and perplexity of the trigram model using Laplace smoothing
print("perplexity: {0:.3f}".format(perpl))

Cross Entropy: 9.686
perplexity: 823.750


### Question 3

In [ ]:
 #Function to retrieve key from value of input from given dictionary 
def get_key(my_dict, val):
    for key, value in my_dict.items():
         if val == value:
             return key

In [ ]:
# Generalized sentence correction using beam search for bigrams ( c is the number of possible words/The higher the l the more context aware the model/The higher the r the more we focus on correct spelling)
def bi_beam_search_ssj(sente,k = 30, c = 3,l = 0.7, r = 0.3): 
  sente = sente.lower() #Lowercasing our data
  ws = word_tokenize(sente)
  ws = ['<start>'] + ws + ['<end>']  # we create sentences for the entire corpus that begin and end with the special tokens (<start>, <end>). We use only one end token because the moment we find the first end token
  pw = ['<start>'] 
  dc = dict()
  dc['<start>'] = [0] # dictionary initialization 
  final = []
  for idx in range(len(ws)-2):
    dist_temp = []
    temp = []
    for word in vocabtrain: #  we keep the best k words(Lowest levenstein distance between target word and vocabulary word)
      dist_temp.append((ld(word,ws[idx+1]),word))
    distance_srt=dist_temp
    distance_srt.sort()
    distance_k=distance_srt[:k] #Get the K first possible words
    for p in pw:
      for dk in distance_k:
        temp.append((dc[p][0] -l*biprob(p,dk[1])-r*(math.log2(1/(1+ld(dk[1],ws[idx+1])))),(dk[1],p)))
    tmp_srt = temp
    tmp_srt.sort()
    tmp_k= tmp_srt[:c] # it keeps the three best TOTAL probabilities ( not only the three best bigrams of the specific level)
    for tk in tmp_k:
      dc[tk[1]] = [tk[0]] # the value of added probabilities of the specific sentence so far
    pw = [t[1] for t in tmp_k]
  ex = dc[pw[0]][0] # starting arbitrary value of ex for the following comparison
  for wor in pw:
    if dc[wor][0] <= ex: # trying to find the min value ( because we have negative values)
      ex = dc[wor][0]
      a = dc[wor]
  key = get_key(dc, a)
  for i in range(idx+1):
    final.append(key[0])
    key=key[1] # the tuple of the most probable sum of bigrams 
  final_sent = ' '.join(f for f in final[::-1]) # joins all the words to form the final sentnece 
  return  final_sent

In [ ]:
bi_beam_search_ssj('I is allxx')

'it is alex'

In [ ]:
# Sentence correction using beamsearch technique on trigrams
def tri_beam_search(x,k=30,l = 0.7,r = 0.3): # ( k is the number of words stored in each level/The higher the l the more context aware the model/The higher the r the more we focus on correct spelling)
  x=x.lower()  #Lowercasing our data
  ws = word_tokenize(x)
  ws = ['<start>']+['<start>'] + ws + ['<end>']  # Creating sentences for the entire corpus that begin and end with the special tokens (<start>, <end>). We only use one end token because the moment we find it we know this is the end of the sentence.(No need for a second token)
  possible_word = ['<start>']+['<start>']
  final= []
  for idx in range(len(ws)-3):
    temp = [] 
    distance=[]
    for word in vocabtrain: # Storing the best k words(Lowest levenstein distance between target word and vocabulary word)
      distance.append((ld(word,ws[idx+2]),word))
    distance_srt=distance
    distance_srt.sort()
    distance_k=distance_srt[:k]
    for elem in distance_k: # Using the above k words we calculate the aggregate probability using both levenstein distance and the specific bigrams probability
      temp.append((-l*triprob(possible_word[0],possible_word[1],elem[1])-r*(math.log2(1/(1+ld(elem[1],ws[idx+2])))),elem[1]))  
    tmp_srt = temp
    tmp_srt.sort()
    tmp_k= tmp_srt[:1] 
    a=possible_word[1]
    possible_word[0]=a
    possible_word[1]= tmp_k[0][1]
    final.append(possible_word[1]) #Final sentence prediction
    final_pred = ' '.join(f for f in final)
  return final_pred

In [ ]:
tri_beam_search('Aftre all teese years you wouldd like to meeet')

'after all these years you would like to eat'